In [8]:
from openai import OpenAI

key_file = open('.key', 'r')
OPENAI_KEY = key_file.read()
key_file.close()
client = OpenAI(api_key=OPENAI_KEY)

In [7]:
def chunk_text(text, max_chars=3000):
    """Split text into chunks of approximately max_chars characters"""
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        if current_length + len(word) + 1 > max_chars and current_chunk:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = len(word)
        else:
            current_chunk.append(word)
            current_length += len(word) + 1
            
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks

def emb_text(text):
    return (
        client.embeddings.create(input=text, model="text-embedding-3-small")
        .data[0]
        .embedding
    )

In [26]:
# read all files from ./_data/clean/
import os
import json
from tqdm import tqdm

documents = []

def save_documents(documents):
    with open('./_data/processed_documents.json', 'w', encoding='utf-8') as f:
        json.dump(documents, f, ensure_ascii=False, indent=2)

for file in tqdm(os.listdir('./_data/clean/'), desc="Processing files"):
    with open(f'./_data/clean/{file}', 'r', encoding='utf-8') as f:
        text = f.read()
        # Split text into chunks
        chunks = chunk_text(text)
        
        # Process each chunk separately
        for i, chunk in enumerate(tqdm(chunks, desc="Processing chunks")):
            chunk_id = f"{file}_chunk_{i}"
            documents.append({
                "id": chunk_id,
                "text": chunk,
                "metadata": {
                    "source": file,
                    "chunk_index": i
                },
                "embedding": emb_text(chunk)
            })
            print(f"Processed {chunk_id}")
        
    save_documents(documents)

print("Finished processing documents")


Processing files:   0%|          | 0/11 [00:00<?, ?it/s]

Processed aktualnosci_.txt_chunk_0


Processing files:   9%|▉         | 1/11 [00:00<00:08,  1.14it/s]

Processed aktualnosci_.txt_chunk_1


Processed dzialalnosc-wsei_.txt_chunk_0


...

Processed zaklady-naukowo-dydaktyczne_zaklad-informatyki_.txt_chunk_4


Processed zaklady-naukowo-dydaktyczne_zaklad-informatyki_.txt_chunk_5


Processed zaklady-naukowo-dydaktyczne_zaklad-informatyki_.txt_chunk_6


Processing files: 100%|██████████| 11/11 [00:12<00:00,  1.10s/it]

Processed zaklady-naukowo-dydaktyczne_zaklad-informatyki_.txt_chunk_7
Finished processing documents


In [5]:
import chromadb

chroma_client = chromadb.PersistentClient(path="./chroma_db")

collection = chroma_client.get_or_create_collection("wsei_data")


In [28]:
document = json.load(open('./_data/processed_documents.json', 'r', encoding='utf-8'))


for doc in tqdm(document, desc="Adding documents to collection"):
    collection.add(
        documents=[doc["text"]],
        metadatas=[doc["metadata"]],
        embeddings=[doc["embedding"]],
        ids=[doc["id"]]
    )

print("Finished adding documents to collection")

Adding documents to collection: 100%|██████████| 34/34 [00:00<00:00, 55.19it/s]

Finished adding documents to collection


In [14]:
query = "jaka jest oferta edukacyjna na wyższej szkole ekonomii i informatyki"
query_embedding = emb_text(query)

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=2
)

print(results)


{'ids': [['studenci_.txt_chunk_1', 'kandydaci_studia-i-stopnia_informatyka-stosowana_.txt_chunk_0']], 'embeddings': None, 'documents': [["otwiera przed Tobą nowe horyzonty nauki! Opłaty za studia Chcesz wiedzieć, jakie korzyści płyną z inwestycji w edukację? Sprawdź zakładkę opłat za studia i dowiedz się więcej o atrakcyjnych warunkach finansowania Twojego rozwoju na WSEI! Certyfikaty W Wyższej Szkole Ekonomii i Informatyki w Krakowie wierzymy, że edukacja to nie tylko zdobywanie wiedzy, ale przede wszystkim jej praktyczne potwierdzenie. Biblioteka W celu otrzymania kodu dostępu należy skontaktować się on line z pracownikiem biblioteki. Samorząd studentów Studiowanie nie polega wyłącznie na zdobywaniu wiedzy naukowej – to okres nabywania szeregu nowych, życiowych doświadczeń. Wobec studentów chętnych do podjęcia współpracy, staramy się o możliwie odpowiednie środowisko osobistego rozwoju. Koła naukowe Wyższa Szkoła Ekonomii i Informatyki w Krakowie aktywnie uczestniczy w rozwoju zainte

In [3]:
def answer_query(query):
    query_embedding = emb_text(query)

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=10
    )

    context = ""

    for doc in results['documents'][0]:
        context += f"{doc} \n"

    open('./_data/context.txt', 'w', encoding='utf-8').write(context)
    
    asistant_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Human: You are an AI assistant. You are able to find answers to the questions from the contextual passage snippets provided."},
            {"role": "user", "content": 
            f"""
            Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
            <context>
            {context}
            </context>
            <question>
            {query}
            </question>
             """
            },
        ]
    )
    
    return asistant_response.choices[0].message.content

In [10]:
answer_query("Jak mogę się skontaktować z WSEI?")

'Na nadchodzących wydarzeniach w Wyższej Szkole Ekonomii i Informatyki (WSEI) planowane są m.in.:\n\n1. Hackathon WSEICRAFT, w ramach którego zespoły mają za zadanie zaprojektować jedną grę komputerową w ciągu 24 godzin.\n2. Hackathon Sheep Your Hack, który również jest 24-godzinnym maratonem programowania, mającym na celu stworzenie minimalnej wersji produktu (MVP) aplikacji biznesowej.\n3. Konferencja Smart City, organizowana w celu podniesienia poziomu zaawansowania samorządów we wdrażaniu zasad zrównoważonego rozwoju.\n\nTe wydarzenia są częścią bogatej oferty uczelni, która stawia na innowacyjność, rozwój i współpracę z różnymi instytucjami.'